In [4]:
!pip install easyocr

  Using cached easyocr-1.7.0-py3-none-any.whl (2.9 MB)
  Using cached torch-2.0.1-cp311-cp311-win_amd64.whl (172.3 MB)
  Using cached torchvision-0.15.2-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached opencv_python_headless-4.8.0.76-cp37-abi3-win_amd64.whl (38.0 MB)
  Using cached scipy-1.11.2-cp311-cp311-win_amd64.whl (44.0 MB)
  Using cached scikit_image-0.21.0-cp311-cp311-win_amd64.whl (22.8 MB)
  Using cached python_bidi-0.4.2-py2.py3-none-any.whl (30 kB)
  Using cached pyclipper-1.3.0.post4-cp311-cp311-win_amd64.whl (93 kB)
  Using cached ninja-1.11.1-py2.py3-none-win_amd64.whl (313 kB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
                                              0.0/5.7 MB ? eta -:--:--
                                              0.0/5.7 MB 1.4 MB/s eta 0:00:05
                                              0.1/5.7 MB 656.4 kB/s eta 0:00:09
                                              0.1/5.7 MB 845.5 kB/s eta 0:00:07
     -                            


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [133]:
!pip install mysql


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [134]:
!pip install mysql-connector-python


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
import mysql.connector
my_sql_connection = mysql.connector.connect(host ='localhost',user='root',password='1234')
mycursor = my_sql_connection.cursor()

In [67]:
mycursor.execute('use imagereader;')

In [140]:
mycursor.execute('create table imagedata(Name varchar(50),Position varchar(50),Website varchar(50),Mobile varchar(50),Email varchar(50),Address varchar(50),District varchar(50),State varchar(50),Pincode varchar(50),Company_Name varchar(50))')

ProgrammingError: 1050 (42S01): Table 'imagedata' already exists

In [1]:
import easyocr
import re
import pandas as pd
import streamlit as st
import numpy as np
reader = easyocr.Reader(['en'])
results = reader.readtext('Images/4.png')
details={
    'Name':[],
    'Position' :[],
    'Website' :[],
    'Mobile' :[],
    'Email' :[],
#     'Address' : [],
    'District' :[],
    'State':[],
    'Pincode':[],
    'Company_Name':[]
}
for index,result in enumerate(results):
    if '@' in result[1] and '.com' in result[1]:
        details['Email'].append(result[1])
    elif '-' in result[1]:
        details['Mobile'].append(result[1])
    elif 'www' in result[1].lower() or 'www.' in result[1].lower():
        details['Website'].append(result[1])
    elif index == 0:
        details['Name'].append(result[1])
    elif index == 1:
        details['Position'].append(result[1])
    elif re.findall("[a-zA-Z]{9} +[0-9]", result[1]):
        details["Pincode"].append(result[1][10:])
    elif len(result[1])>=6 and result[1].isdigit():
        details["Pincode"].append(result[1])
    state_extraction = re.findall("[a-zA-Z]{9} +[0-9]", result[1]) 
    state_extraction_1 = re.findall("^[0-9].+, ([a-zA-Z]+);", result[1])
    if state_extraction:
        details["State"].append(result[1][:9])
    elif state_extraction_1:
        details["State"].append(result[1].split()[-1])
    if len(details["State"]) == 2:
        details["State"].pop(0)
    elif index == len(results) -1:
        details["Company_Name"].append(result[1])
#     address = re.findall("'^[0-9].+, [a-zA-Z]+'", result[1]) or re.findall(".+St, ([a-zA-Z]+).+", result[1]) or re.findall('^[E].*',result[1])
#     if address:
#         details["Address"].append(result[1].split()[:-2])
#     district = re.findall(".+St,, ([a-zA-Z]+).+", result[1]) or re.findall(".+St, ([a-zA-Z]+)+", result[1]) or re.findall('^[E].*',result[1])
    match1 = re.findall('.+St , ([a-zA-Z]+).+', result[1])
    match2 = re.findall('.+St,, ([a-zA-Z]+).+', result[1])
    match3 = re.findall('^[E].*',result[1])
    if match1:
        details["District"].append(match1[0])
    elif match2:
        details["District"].append(match2[0])
    elif match3:
        details["District"].append(match3[0])
rows = []
for i in range(len(details['Name'])):
    row = {
        'Name': details['Name'][i],
        'Position': details['Position'][i],
        'Website': details['Website'][i],
        'Mobile': details['Mobile'][i],
        'Email': details['Email'][i],
#         'Address': details['Address'][i],
        'District': details['District'][i],
        'State': details['State'][i],
        'Pincode': details['Pincode'][i],
        'Company_Name': details['Company_Name'][i]
    }
    rows.append(row)
print(rows)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[{'Name': 'REVANTH', 'Position': 'Marketing Executive', 'Website': 'wWW.CHRISTMAS.com', 'Mobile': '+91-456-1234', 'Email': 'hello@CHRISTMAS.com', 'District': 'HYDRABAD', 'State': 'TamilNadu;', 'Pincode': '600001', 'Company_Name': 'Restaurant'}]


In [279]:
for data in rows:
    sql_insert = f'INSERT INTO imagedata(Name,Position,Website,Mobile,Email,Address,District,State,Pincode,Company_Name) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    mycursor.execute(sql_insert,tuple(data.values()))
    my_sql_connection.commit()

In [69]:
mycursor.execute('Select Name from imagedata')
result  = mycursor.fetchall()
cards ={}
for data in result:
    cards[data[0]] = data[0]
cards

{'Selva': 'Selva', 'Amit kumar': 'Amit kumar', 'SANTHOSH': 'SANTHOSH'}